In [1]:
import blenderproc as bproc
import argparse
import numpy as np 


Using blender in C:\Users\gaura\blender\blender-3.3.1-windows-x64


Traceback (most recent call last):
  File "C:\Users\gaura\anaconda3\Scripts\blenderproc-script.py", line 33, in <module>
    sys.exit(load_entry_point('blenderproc', 'console_scripts', 'blenderproc')())
  File "c:\users\gaura\blenderproc\blenderproc\command_line.py", line 165, in cli
    SetupUtility.check_if_setup_utilities_are_at_the_top(path_src_run)
  File "c:\users\gaura\blenderproc\blenderproc\python\utility\SetupUtility.py", line 413, in check_if_setup_utilities_are_at_the_top
    raise RuntimeError(f"The given run script does not exist: {path_to_run_file}")
RuntimeError: The given run script does not exist: examples/thesis/project/main.py


In [ ]:

parser = argparse.ArgumentParser()
parser.add_argument('scene', help="Path to the scene.obj file, should be examples/resources/scene.obj")
parser.add_argument('output_dir', help="Path to where the final files, will be saved, could be examples/basics/basic/output")
args = parser.parse_args()

bproc.init()

# load the objects into the scene
objs = bproc.loader.load_obj(args.scene)



In [ ]:
# define a light and set its location and energy level
light = bproc.types.Light()
light.set_type("POINT")
light.set_location([5, -5, 5])
light.set_energy(1000)



In [ ]:
# define the camera resolution
bproc.camera.set_resolution(512, 512)

# Find point of interest, all cam poses should look towards it
poi = bproc.object.compute_poi(objs)
# Sample five camera poses
for i in range(5):
    # Sample random camera location above objects
    location = np.random.uniform([-400, -200, 500], [400, 200, 500])
    # Compute rotation based on vector going from location towards poi
    rotation_matrix = bproc.camera.rotation_from_forward_vec(poi - location, inplane_rot=np.random.uniform(-0.7854, 0.7854))
    # Add homog cam pose based on location an rotation
    cam2world_matrix = bproc.math.build_transformation_mat(location, rotation_matrix)
    bproc.camera.add_camera_pose(cam2world_matrix)


In [ ]:
# activate normal and depth rendering
bproc.renderer.enable_normals_output()
bproc.renderer.enable_depth_output(activate_antialiasing=False)


In [ ]:

# render the whole pipeline
data = bproc.renderer.render()

# write the data to a .hdf5 container
bproc.writer.write_hdf5(args.output_dir, data)